1. Download/Load SP500 stocks prices data

In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import  numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [50]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-01-01'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%***********************]  503 of 503 completed

4 Failed downloads:
['SW', 'AMTM', 'SOLV', 'GEV']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2016-01-03 00:00:00 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1451797200, endDate = 1704085200")')


Price                              adj close       close        high  \
date                      ticker                                       
2016-01-04 00:00:00+00:00 A        38.664230   40.689999   41.189999   
                          AAPL     24.323219   26.337500   26.342501   
                          ABBV     42.991688   57.610001   58.320000   
                          ABT      38.193447   42.930000   44.049999   
                          ACGL     22.950001   22.950001   23.219999   
...                                      ...         ...         ...   
2023-12-29 00:00:00+00:00 XYL     114.360001  114.360001  114.680000   
                          YUM     130.660004  130.660004  131.250000   
                          ZBH     121.699997  121.699997  122.400002   
                          ZBRA    273.329987  273.329987  276.309998   
                          ZTS     197.369995  197.369995  198.009995   

Price                                    low        open       volume  
date                      ticker                                       
2016-01-04 00:00:00+00:00 A        40.340000   41.060001    3287300.0  
                          AAPL     25.500000   25.652500  270597600.0  
                          ABBV     56.400002   58.060001   10308400.0  
                          ABT      42.500000   43.939999   12227500.0  
                          ACGL     22.616667   22.940001    1220700.0  
...                                      ...         ...          ...  
2023-12-29 00:00:00+00:00 XYL     113.930000  114.089996     698900.0  
                          YUM     130.210007  130.410004    1196800.0  
                          ZBH     121.239998  121.459999     849600.0  
                          ZBRA    272.769989  274.730011     251300.0  
                          ZTS     196.250000  196.679993    1007200.0  

[984053 rows x 6 columns]

2. Calculate features and technical indicators for each stock.
* Garman-Klass Volatility
* RSI
* Bollinger Bands
* ATR
* MACD
* Dollar Volume

\begin{equation}
\text{Garman-Klass Volatility} = \frac{(\ln(\text{High}) - \ln(\text{Low}))^2}{2} - (2\ln(2) - 1)(\ln(\text{Adj Close}) - \ln(\text{Open}))^2
\end{equation}

In [52]:
df.columns = df.columns.str.lower()

df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])

df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price                              adj close       close        high  \
date                      ticker                                       
2016-01-04 00:00:00+00:00 A        38.664230   40.689999   41.189999   
                          AAPL     24.323219   26.337500   26.342501   
                          ABBV     42.991688   57.610001   58.320000   
                          ABT      38.193447   42.930000   44.049999   
                          ACGL     22.950001   22.950001   23.219999   
...                                      ...         ...         ...   
2023-12-29 00:00:00+00:00 XYL     114.360001  114.360001  114.680000   
                          YUM     130.660004  130.660004  131.250000   
                          ZBH     121.699997  121.699997  122.400002   
                          ZBRA    273.329987  273.329987  276.309998   
                          ZTS     197.369995  197.369995  198.009995   

Price                                    low        open       volume  \
date                      ticker                                        
2016-01-04 00:00:00+00:00 A        40.340000   41.060001    3287300.0   
                          AAPL     25.500000   25.652500  270597600.0   
                          ABBV     56.400002   58.060001   10308400.0   
                          ABT      42.500000   43.939999   12227500.0   
                          ACGL     22.616667   22.940001    1220700.0   
...                                      ...         ...          ...   
2023-12-29 00:00:00+00:00 XYL     113.930000  114.089996     698900.0   
                          YUM     130.210007  130.410004    1196800.0   
                          ZBH     121.239998  121.459999     849600.0   
                          ZBRA    272.769989  274.730011     251300.0   
                          ZTS     196.250000  196.679993    1007200.0   

Price                             garman_klass_vol        rsi    bb_low  \
date                      ticker                                          
2016-01-04 00:00:00+00:00 A              -0.001179        NaN       NaN   
                          AAPL           -0.000565        NaN       NaN   
                          ABBV           -0.034315        NaN       NaN   
                          ABT            -0.006947        NaN       NaN   
                          ACGL            0.000346        NaN       NaN   
...                                            ...        ...       ...   
2023-12-29 00:00:00+00:00 XYL             0.000019  75.492358  4.653088   
                          YUM             0.000030  58.710389  4.824633   
                          ZBH             0.000044  68.324469  4.756323   
                          ZBRA            0.000073  67.914673  5.415905   
                          ZTS             0.000035  66.737233  5.186448   

Price                               bb_mid   bb_high       atr      macd  \
date                      ticker                                           
2016-01-04 00:00:00+00:00 A            NaN       NaN       NaN       NaN   
                          AAPL         NaN       NaN       NaN       NaN   
                          ABBV         NaN       NaN       NaN       NaN   
                          ABT          NaN       NaN       NaN       NaN   
                          ACGL         NaN       NaN       NaN       NaN   
...                                    ...       ...       ...       ...   
2023-12-29 00:00:00+00:00 XYL     4.707545  4.762003 -0.345889  1.909542   
                          YUM     4.862793  4.900952 -0.073239  0.705235   
                          ZBH     4.786051  4.815780 -1.054422  0.940596   
                          ZBRA    5.546306  5.676707 -0.154931  1.577769   
                          ZTS     5.259449  5.332449  0.420520  2.133384   

Price                             dollar_volume  
date                      ticker                 
2016-01-04 00:00:00+00:00 A          127.100924 